# Setup

Clone GitHub [repository](https://github.com/Sydney-Informatics-Hub/PIPE-4011-EM-CV), install [dependencies](https://github.com/Sydney-Informatics-Hub/PIPE-4011-EM-CV/blob/master/requirements.txt) and check PyTorch and GPU.

In [ ]:
!git clone https://github.com/Sydney-Informatics-Hub/PIPE-4011-EM-CV  # clone

%cd PIPE-4011-EM-CV
%pip install -qr requirements.txt
# install yolo dependencies
%cd yoloseg
%pip install -qr requirements.txt

In [ ]:
import sys
import os
import torch
import utils
import subprocess
display = utils.notebook_init()  # checks

In [ ]:
## Add local modules to path
sys.path.append('../tools')
import yolo2mask
import image2tiles
import merge_tiles

### Input Settings

In [ ]:
# Path to input images
path_images = '../testdata/sample_highres'

# Path to results
path_results = '../results_mito'

# Path to the trained model
path_model = 'models/yolov5m_seg_mito_lipids.pt'

# tile sizes for inference (dependent on input image size and size of objects of interest)
tile_size = 1280

## Prepare Data

Slice images into tiles

In [ ]:
# List of images to process
accepted_formats = ["png", "jpg", "jpeg", "tif", "tiff"]
image_list = [f for f in os.listdir(path_images) if f.split(".")[-1].lower() in accepted_formats]
basenames = [f.split(".")[0] for f in image_list]
print("Found {} images to process.".format(len(image_list)))

In [ ]:
path_tiles = os.path.join(path_results, 'tiles')
image2tiles.tile_images_in_folder(path_images, tile_width=tile_size, tile_height=tile_size, max_overlap_ratio=0, outpath=path_tiles)

## Segmentation Predictions

Use finetuned YOLO segmentation model to generate segmentation labels.

In [ ]:
outpat_pred = os.path.join(path_results, "predict-seg")
cmd = f"python predict.py --weights {path_model} --source {path_tiles} --project {outpat_pred} --img 640 --conf 0.4 --line-thickness 3 --save-txt"
subprocess.run(cmd.split())

## Generate image masks

Convert YOLO ouptut labels (.txt files) to image masks

In [ ]:
path_labels = os.path.join(outpat_pred, 'exp/labels')
path_masks = os.path.join(path_results, 'masks')
yolo2mask.convert_to_mask(path_labels, path_tiles, outpath = path_masks, cat_id=0, format_out="jpg")

## Merge masks

Merge mask tiles back to fullsize image

In [ ]:
outpath_merged = os.path.join(path_results, 'masks_merged')
os.makedirs(outpath_merged, exist_ok=True)
for basename in basenames:
    merge_tiles.merge(path_masks, basename, output_file= os.path.join(outpath_merged, f'{basename}_mask_mito.jpg'))